# Putting Redux (CT 8)

For this challenge task, you'll be asked to revisit the lag putting problem we did early on in the term.  But now we have the tools to create a much more realistic model: we can deal with the continuous changes in the slope of the green, we know how to deal with vectors, and we have a range of optimization tools at our disposal.

<br>

---

In [ ]:
#@title
# Import necessary libraries
from os.path import basename, exists
from os import mkdir

def download(url,folder):
    filename = folder + basename(url)
    if not exists(folder):
        mkdir(folder)
    # fetches the file at the given url if it is not already present
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

download('https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Notebooks/'
        + 'ModSimPy_Functions/modsim.py', 'ModSimPy_Functions/')

from ModSimPy_Functions.modsim import *
import pandas as pd
import numpy as np

---

<br>

## The Challenge

Your goal is to create a simulation that can determine the velocity and direction of a putt that will stop at the hole.  The initial location of the ball and the location of the hole should be system parameters for an individual simulation (and should be adjustable for a later simulation).

<br> 

The only predetermined parameter that you will need is the coefficient of rolling friction ($\mu = 0.15$).  If you choose to incorporate drag, you can assume that the coefficient of drag is $C_d = 0.7$, the radius of the ball is $r = 2.16 cm$, the mass of the ball is $0.046 kg$, and the density of air is $1.3 kg/m^3$.

## The Green

The details for the green are included here.  As you will see, the green data is formatted in a way that will allow you to access the slope of the green at any given coordinate point.  To help you visualize the green, here are two plots: the first of the elevation on the green, and the second of the slope of the green (You can look at this code to see where this is coming from, but don't necessarily need to).

<br>

I've included a second "green" here with a very simple topography that you might use to test your simulation, since its much easier to guess how the ball should respond to this simpler surface.  To use the "test green", just comment out the first `filename` and uncomment the second one.

In [ ]:
#@title
from scipy.interpolate import interpn
from numpy import gradient
import matplotlib.pyplot as plt

# Upload elevation data for the green and put data in an array
# Switch to the second filename to do simple testing of your simulation
filename = 'https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Data/CT8_golf_green_data.xlsx'
#filename = 'https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Data/CT8_golf_green_data_test.xlsx'
data = pd.read_excel(filename, header=0, index_col=0)
green = np.array(data)

# Calculate the gradients at the green so that they can
# be accessed using coordinate values
grad = gradient(green)
grad[0] = np.flip(np.rot90(grad[0]),1)
grad[1] = np.flip(np.rot90(grad[1]),1)
grad = np.flip(grad)

# Plot a topographic map of the green
fig, (ax_topo, ax_slope) = plt.subplots(nrows=1, ncols=2, figsize=(14,10))
cs = ax_topo.contour(np.array(data.columns), np.array(data.index), green, levels=15)
ax_topo.clabel(cs,colors='black', fmt = '%1.3f');
ax_topo.set(title='Elevation Map of Green', xlabel='X-coordinate', ylabel='Y-coordinate');

# Plot the slope of the green (arrows pointing downhill)
x, y = np.meshgrid(np.array(data.index), np.array(data.columns))
ax_slope.quiver(y,x,-grad[0],-grad[1]);
ax_slope.set(title='Slope Map of Green', xlabel='X-coordinate');

Note that both the coordinate values and the elevation values are in meters.

<br>

The NumPy array `grad` (created in the cell above) holds the gradient values at a set of defined coordinate points, which are in regular grid with a spacing of 5 ft (1.52 m).  For instance, we can access the slope of the green at the coordinate point (1.52 m,1.52 m) by doing the following:


In [ ]:
print(grad[0][1][1], grad[1][1][1])

Make sure you understand what these values mean by looking at the contour map above (and note that the slope map on the right shows the *downhill* slope, so the signs are reversed on the plot).  But `grad` by itself has a limited usefulness:

<br>

* Its arguments refer to index numbers, rather than distances
* More importantly, it can provide the slope only at a given set of data points, but you will need to access the slope at any plot on the green. So we need some interpolation, right?! We can find the slope at any point by interpolating between known points. 

<br> 

Since we need to interpolate in two dimensions, the tool we used earlier (`interp1d`) won't work.  Instead, we can use the SciPy function `interpn`, which interpolates an x- and y- component of the slope for any point on the green.  This code returns the x-component of the slope at the point $(1.5 ~m, 1.5 ~m)$.

In [ ]:
# Create an array that describes the location of known data points
points = (np.array(data.columns), np.array(data.index))

# Find the interpolated slope at the given point (here, (1.5, 1.5))
# The 'bounds_error' tells the function not to produce an error
# when the ball leaves the green
slope_array = interpn(points,grad[0],[1.5,1.5], bounds_error=False)

# 'interpn' returns an array of length 1; for convenience, the next line takes the value
# out of the array and creates a simple float (decimal) number
inter_slope = slope_array[0]
inter_slope

`points` creates an array of the known points: you'll need to keep that in your `params` object.  `interpn` returns the x-component of the slope if `grad[0]` is entered as the second argument; if `grad[1]` is the second argument, the return value is the y-component of the slope.

## How to Proceed

Here are some things to start with:

<br>

* To understand the physics of a ball rolling on uneven ground, you might look back at Challenge Task 1.  

* Create a set of `params`, and a `make_system` function.  What values will you need to run a particular simulation?

* To understand the structure of the code and how to use vectors, you could start by looking at the baseball notebooks (3.9.1-3.9.3).  The code in 3.9.1 would be a good place to start. 

* Look at the code map of the baseball notebooks to understand the different parts of the code you'll need for the optimization process.

<br>

The best advice I can give is to iterate slowly through the process:

* In your first iteration, only include frictional force, and ignore the $cos~ \theta$ part of the term (the slopes are small enough that $cos ~\theta \approx 1.0$).  The ball will roll straight (gravity causes the rolling ball to curve), and should stop.

* Test your first iteration.  If it works, add the cosine term to frictional force.  

* If that works, add the gravitational force term.  

* Finally, add drag if you can.

* Only then should you start to think about optimization.  Start by creating a function that you want to minimize by changing one parameter (like the `height_func` in the baseball notebook).  You'll eventually need two of these (one for velocity, and one for angle).



## Some specific tips

* You are going to need an `event_func` that stops the simulation if the ball goes off the grid.  Note that `points` contains the boundary edge value: for example, `points[1][-1]` will produce the largest y coordinate value on the green.

* The dimensions you are tracking will be in the horizontal direction.  You do not need to track the vertical direction, as the differences in elevation are quite small and the ball is always on the surface.

* Include `points` and `grad` in your `params` and `system`, as you will need to access these any time you use `interpn`.  

* As in the baseball notebook 3.9.3, you will need to solve for both the best angle and best velocity, so you'll need to nested loops (see the baseball code map).  In this case, you are probably better off using `minimize_scalar` for both loops; we want to minimize the final distance to the hole, and this value is never negative, so `root_scalar` will not work very well.

* Finally, here are a couple functions that we used in the baseball notebooks that you might find useful, and a fancier `plot_trajectory` that overlays the path of the ball onto the contour map:

In [ ]:
def angle_to_components(mag,angle):
    theta = np.deg2rad(angle)
    x = mag * np.cos(theta)
    y = mag * np.sin(theta)
    return pd.Series(dict(x=x,y=y),dtype=float)

def plot_trajectory(results,**options):
    x = results.x.values
    y = results.y.values
    x_vs_y = pd.Series(data=y,index=x)
    x_vs_y.plot(xlabel='x position (m)', ylabel='y position (m)',
                figsize=[6,9], xlim=[0,12.2],ylim=[0,21.3],**options)
    
# Note: to use this function, you will need to call the coordinates of your
# hole location 'xh' and 'yh' (or you can change this function to match
# your variable names)    
def plot_trajectory_contour(results,system):
    x = results.x.values
    y = results.y.values
    x_vs_y = pd.Series(data=y,index=x)
    y_arr = ([system['yh']])
    x_arr = ([system['xh']])
    hole_loc = pd.Series(data=y_arr,index=x_arr)

    fig, ax = plt.subplots(figsize=(6,10))
    cs = ax.contour(np.array(data.columns), np.array(data.index), green, levels=15)
    ax.clabel(cs,colors='black', fmt = '%1.3f');
    ax.set(title='Optimized Lag Putt', xlabel='X-coordinate', ylabel='Y-coordinate');

    x_vs_y.plot(xlabel='x position (m)',
             ylabel='y position (m)',figsize=[6,9], xlim=[0,12.2],ylim=[0,21.3])
    hole_loc.plot(style = 'o')